In [ ]:
! pip install simpletransformers

In [ ]:
# https://stackoverflow.com/questions/56850711/importerror-please-install-apex-from-https-www-github-com-nvidia-apex-to-use

import os, sys, shutil
import time
import gc
from contextlib import contextmanager
from pathlib import Path
import random
import numpy as np, pandas as pd
from tqdm import tqdm, tqdm_notebook

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

USE_APEX = True

if USE_APEX:
            with timer('install Nvidia apex'):
                # Installing Nvidia Apex
                os.system('git clone https://github.com/NVIDIA/apex; cd apex; pip install -v --no-cache-dir' + 
                          ' --global-option="--cpp_ext" --global-option="--cuda_ext" ./')
                os.system('rm -rf apex/.git') # too many files, Kaggle fails
                from apex import amp

[install Nvidia apex] done in 461 s


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_raw = pd.read_csv('full_train.csv')
test_raw = pd.read_csv('full_test.csv')

In [ ]:
train_raw

,Headline,Body ID,Stance,articleBody
0,Angry mob hacks off alleged rapist's genitals ...,1709,discuss,A MAN accused of trying to rape a teenage girl...
1,ISIS fighters ‘have contracted Ebola’,2180,discuss,No suspected cases of Ebola have been found in...
2,‘The cub of Baghdadi': ISIS reports its younge...,299,discuss,Young boy is seen holding guns in images circu...
3,Reports Isis fighters have contracted Ebola ar...,1689,discuss,Islamic State fighters are no match for Ebola....
4,Surreal Photos of Fisherman’s Jaw-Dropping Cat...,1183,unrelated,"""Multiple unconfirmed sources"" report that Hai..."
...,...,...,...,...
26849,"Source: Pot in NYC may soon net just a ticket,...",139,unrelated,Judd Nelson rebuffs Internet rumors that he di...
26850,"No, Banksy Didn't Create That 'Charlie Hebdo' ...",2526,unrelated,"Update: Since the publication of this article,..."
26851,Watch: ISIS Claims to Have Captured U.S. Airdr...,1361,discuss,"In a new video, ISIS shows American-made weapo..."
26852,Seven girls fall pregnant after five day schoo...,2504,agree,Seven girls aged between 13 and 14 have fallen...


In [ ]:
train = pd.DataFrame(columns=['text_a', 'text_b', 'labels'])
train['text_a'] = train_raw['Headline']
train['text_b'] = train_raw['articleBody']

In [ ]:
stances = train_raw['Stance']
stances.replace('agree', 0,True)
stances.replace('disagree', 1, True)
stances.replace('discuss', 2, True)
stances.replace('unrelated', 3, True)
stances
train['labels'] = stances

In [ ]:
stances = test_raw['Stance']
stances.replace('agree', 0,True)
stances.replace('disagree', 1, True)
stances.replace('discuss', 2, True)
stances.replace('unrelated', 3, True)
stances

0        3
1        3
2        3
3        3
4        3
        ..
25408    0
25409    2
25410    1
25411    1
25412    0
Name: Stance, Length: 25413, dtype: int64

In [ ]:
test = pd.DataFrame(columns=['text_a', 'text_b'])

test['text_a'] = test_raw['Headline']
test['text_b'] = test_raw['articleBody']
# test['labels'] = test_raw['label']

In [ ]:
train

,text_a,text_b,labels
0,Angry mob hacks off alleged rapist's genitals ...,A MAN accused of trying to rape a teenage girl...,2
1,ISIS fighters ‘have contracted Ebola’,No suspected cases of Ebola have been found in...,2
2,‘The cub of Baghdadi': ISIS reports its younge...,Young boy is seen holding guns in images circu...,2
3,Reports Isis fighters have contracted Ebola ar...,Islamic State fighters are no match for Ebola....,2
4,Surreal Photos of Fisherman’s Jaw-Dropping Cat...,"""Multiple unconfirmed sources"" report that Hai...",3
...,...,...,...
26849,"Source: Pot in NYC may soon net just a ticket,...",Judd Nelson rebuffs Internet rumors that he di...,3
26850,"No, Banksy Didn't Create That 'Charlie Hebdo' ...","Update: Since the publication of this article,...",3
26851,Watch: ISIS Claims to Have Captured U.S. Airdr...,"In a new video, ISIS shows American-made weapo...",2
26852,Seven girls fall pregnant after five day schoo...,Seven girls aged between 13 and 14 have fallen...,0


In [ ]:
test

,text_a,text_b
0,Ferguson riots: Pregnant woman loses eye after...,A RESPECTED senior French police officer inves...
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,Dave Morin's social networking company Path is...
2,A Russian Guy Says His Justin Bieber Ringtone ...,A bereaved Afghan mother took revenge on the T...
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",Hewlett-Packard is officially splitting in two...
4,Argentina's President Adopts Boy to End Werewo...,An airline passenger headed to Dallas was remo...
...,...,...
25408,The success of the Affordable Care Act is a hu...,"Congressional Republicans, evidently hoping th..."
25409,The success of the Affordable Care Act is a hu...,Did Obamacare work?\n\nIt’s worth reflecting u...
25410,The success of the Affordable Care Act is a hu...,Millions may lose coverage next year if Congre...
25411,The success of the Affordable Care Act is a hu...,"Come November, the grim trudge across the incr..."


In [ ]:
from simpletransformers.classification import ClassificationModel

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=4, args={
    'learning_rate':2e-5,
    'num_train_epochs': 5,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'process_count': 10,
    'evaluate_during_training_steps': False,
    'train_batch_size': 4,
    'eval_batch_size': 4,
    'max_seq_length': 512,
    'fp16': True,
    'early_stopping': True
})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
model.train_model(train)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 512.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 256.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 128.0


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 128.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 128.0



Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 256.0



Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 512.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 512.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 512.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 256.0



Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 256.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 256.0



Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0




In [ ]:
!pwd

/content


In [ ]:
test_ll = list()

for i in range(len(test)):
  test_ll.append([test['text_a'][i], test['text_b'][i]])

In [ ]:
test_ll

In [ ]:
def acc(pred):
  cnt = 0
  for i in range(len(pred)):
    if pred[i] == stances[i]:
      cnt = cnt + 1
  print(cnt / len(pred))

In [ ]:
def createAns(opt, epoch):
  df_answer = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])
  df_answer['Stance'] = opt
  df_answer.replace(0, 'agree',True)
  df_answer.replace(1, 'disagree',True)
  df_answer.replace(2, 'discuss',True)
  df_answer.replace(3, 'unrelated',True)
  df_answer['Headline'] = test_raw['Headline']
  df_answer['Body ID'] = test_raw['Body ID']
  fName = 'answerEp' + str(epoch) + '.csv'
  df_answer.to_csv(fName, index=False, encoding='utf-8')
  return df_answer

In [ ]:
model = ClassificationModel(
    "roberta", "outputs/checkpoint-33570-epoch-5"
)

In [ ]:
predictions, raw_outputs = model.predict(test_ll)

In [ ]:
acc(predictions)

0.9386140951481525


In [ ]:
createAns(predictions, 5)

,Headline,Body ID,Stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated
...,...,...,...
25408,The success of the Affordable Care Act is a hu...,2582,agree
25409,The success of the Affordable Care Act is a hu...,2583,discuss
25410,The success of the Affordable Care Act is a hu...,2584,discuss
25411,The success of the Affordable Care Act is a hu...,2585,discuss


In [ ]:
model4 = ClassificationModel(
    "roberta", "outputs/checkpoint-26856-epoch-4"
)

In [ ]:
predictions4, raw_outputs4 = model4.predict(test_ll)

In [ ]:
createAns(predictions4, 4)

,Headline,Body ID,Stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated
...,...,...,...
25408,The success of the Affordable Care Act is a hu...,2582,discuss
25409,The success of the Affordable Care Act is a hu...,2583,discuss
25410,The success of the Affordable Care Act is a hu...,2584,discuss
25411,The success of the Affordable Care Act is a hu...,2585,discuss


In [ ]:
acc(predictions4)

0.9357808995396057


In [ ]:
model3 = ClassificationModel(
    "roberta", "outputs/checkpoint-20142-epoch-3"
)

In [ ]:
predictions3, raw_outputs3 = model3.predict(test_ll)

In [ ]:
createAns(predictions3, 3)
acc(predictions3)

0.9384566953921222


In [ ]:
model2 = ClassificationModel(
    "roberta", "outputs/checkpoint-13428-epoch-2"
)

In [ ]:
predictions2, raw_outputs2 = model2.predict(test_ll)

In [ ]:
df_2 = createAns(predictions2, 2)
acc(predictions2)

0.9323968047849526


In [ ]:
df_2.isnull().sum().sum()

0

In [ ]:
model1 = ClassificationModel(
    "roberta", "outputs/checkpoint-6714-epoch-1"
)

In [ ]:
predictions1, raw_outputs1 = model1.predict(test_ll)

In [ ]:
df_1 = createAns(predictions1, 1)
acc(predictions1)

0.91575178058474


In [ ]:
df_1.isnull().sum().sum()

0

In [ ]:
predictions[:1000]

array([3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 0,
       0, 3, 2, 3, 3, 2, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3,
       3, 2, 3, 3, 2, 3, 2, 3, 3, 1, 3, 3, 2, 2, 3, 3, 3, 3, 2, 3, 2, 3,
       2, 3, 3, 3, 2, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3,
       3, 2, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 2, 3, 3, 3, 2, 3, 3,
       3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 0, 3, 1,
       3, 0, 3, 3, 3, 3, 3, 3, 2, 2, 3, 1, 3, 3, 3, 3, 3, 2, 2, 3, 3, 2,
       3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 2, 3, 3, 2, 0, 0, 2,
       3, 3, 2, 3, 2, 3, 3, 3, 2, 0, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 3, 3, 3, 3, 3,
       0, 3, 3, 2, 3, 3, 3, 3, 0, 0, 3, 3, 2, 2, 2, 3, 2, 0, 3, 3, 0, 2,
       3, 3, 3, 3, 2, 3, 1, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3,
       3, 3, 2, 2, 3, 3, 0, 2, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [ ]:
test['labels'] = test_raw['label']
test

,text_a,text_b,labels
0,Ferguson riots: Pregnant woman loses eye after...,A RESPECTED senior French police officer inves...,3
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,Dave Morin's social networking company Path is...,3
2,A Russian Guy Says His Justin Bieber Ringtone ...,A bereaved Afghan mother took revenge on the T...,3
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",Hewlett-Packard is officially splitting in two...,3
4,Argentina's President Adopts Boy to End Werewo...,An airline passenger headed to Dallas was remo...,3
...,...,...,...
25408,The success of the Affordable Care Act is a hu...,"Congressional Republicans, evidently hoping th...",0
25409,The success of the Affordable Care Act is a hu...,Did Obamacare work? It’s worth reflecting upo...,2
25410,The success of the Affordable Care Act is a hu...,Millions may lose coverage next year if Congre...,1
25411,The success of the Affordable Care Act is a hu...,"Come November, the grim trudge across the incr...",1


In [ ]:
df_answer = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])

In [ ]:
df_answer

,Headline,Body ID,Stance


In [ ]:
df_answer['Stance'] = predictions

In [ ]:
df_answer.replace(0, 'agree',True)
df_answer.replace(1, 'disagree',True)
df_answer.replace(2, 'discuss',True)
df_answer.replace(3, 'unrelated',True)

In [ ]:
df_answer['Headline'] = test_raw['Headline']
df_answer['Body ID'] = test_raw['Body ID']

In [ ]:
df_answer

,Headline,Body ID,Stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated
...,...,...,...
25408,The success of the Affordable Care Act is a hu...,2582,agree
25409,The success of the Affordable Care Act is a hu...,2583,discuss
25410,The success of the Affordable Care Act is a hu...,2584,unrelated
25411,The success of the Affordable Care Act is a hu...,2585,unrelated


In [ ]:
df_answer.to_csv('answer.csv', index=False, encoding='utf-8')

In [ ]:
df_answer.isnull()

,Headline,Body ID,Stance
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
25408,False,False,False
25409,False,False,False
25410,False,False,False
25411,False,False,False


In [ ]:
df_answer.isnull().sum().sum()

0

In [ ]:
import pickle

In [ ]:
cnt = 0
for i in preds_test:
  if i == 3:
    cnt = cnt + 1

print(cnt)

25413


In [ ]:
print(len(preds_test))

25413
